# Set Up Databases and Populating Tickers and Historical Stock Data

# 1. Setup & Imports

In [ ]:
import sys
import os
import sqlite3
from pathlib import Path
from datetime import datetime, timedelta

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

# Import paths from the config module
from src.config import BASE_DIR, DB_DIR

from src.db_schema import DATABASES
from src.etl.populate_tickers import recreate_database, populate_tickers
from src.etl.populate_prices import ensure_prices_table, populate_prices
from src.utils.db_utils import fetch_all_asset_metadata, fetch_all_asset_prices

# 2. Initialize SQLite Databases

In [ ]:
DB_DIR.mkdir(exist_ok=True)

def setup_databases():
    """Initialize databases and tables based on defined schemas."""
    for db_name, schemas in DATABASES.items():
        db_path = DB_DIR / db_name

        try:
            with sqlite3.connect(db_path) as conn:
                conn.execute("PRAGMA foreign_keys = ON;")
                cursor = conn.cursor()
                for schema in schemas:
                    cursor.execute(schema)
                conn.commit()
                print(f"✅ Tables created/verified in '{db_name}'")
        except sqlite3.Error as e:
            print(f"❌ Error setting up '{db_name}': {e}")

# Run database setup
setup_databases()

# 3. Populate Asset Metadata (Tickers)

In [ ]:
print("\n🔄 Recreating and populating tickers database...")
recreate_database()
populate_tickers()
print("✅ Tickers populated successfully.")

In [ ]:
# View the asset metadata
asset_metadata_df = fetch_all_asset_metadata()

print("DataFrame shape:", asset_metadata_df.shape)
print("DataFrame columns:", asset_metadata_df.columns.tolist())
display(asset_metadata_df.head(20))

In [ ]:
asset_metadata_df['exchange'].unique()

# 4. Populate Historical Stock Prices

In [ ]:
# Ensure the historical prices are populated. 
populate_prices()
print("✅ Historical stock prices populated successfully.")

In [ ]:
# Fetch and display the historical prices
print("\n📊 Fetching and displaying historical stock prices...")
prices_df = fetch_all_asset_prices()

# Display basic info and the first 20 rows
print(f"DataFrame shape: {prices_df.shape}")
print(f"Columns: {prices_df.columns.tolist()}")
display(prices_df.head(20))